In [15]:
!pip install --upgrade pip
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)


In [19]:
!git clone https://github.com/deepset-ai/haystack.git

Cloning into 'haystack'...
remote: Enumerating objects: 32720, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (285/285), done.
remote: Total 32720 (delta 392), reused 436 (delta 296), pack-reused 32135
Receiving objects: 100% (32720/32720), 35.97 MiB | 1.01 MiB/s, done.
Resolving deltas: 100% (24977/24977), done.


In [20]:
!brew install postgresql
!brew install cmake
!brew install rust

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
==> Updated Homebrew from ffa7ee011 to 8c6f31a7a.
Updated 3 taps (homebrew/cask-versions, homebrew/core and homebrew/cask).
==> New Formulae
adr-viewer                               mariadb@10.10
aicommits                                micromamba
aiven-client                             nixpacks
animdl                                   ocmtoc
arttime                                  octobuild
astgen                                   okta-aws-cli
aws-sso-cli                              ol
base16384                                openai-whisper
benerator                                openiked
bwidget                                  papilo
cffi                                     pfetch-rs
clifm                                    phoneinfoga
clipboard                                picotool
conan@1                                  pmtiles
conduit                                  portal
conman                            

In [1]:
!python --version

Python 3.10.2


In [23]:
!cd haystack && pip install -e .

Obtaining file:///Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/data_converter/notebooks/haystack
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached posthog-2.4.0-py2.py3-none-any.whl (36 kB)
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 7.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 16.2 MB/s eta 0:00:0000:0100:01
  Using cached quantulum3-0.8.1-py3-none-any.whl (10.7 MB)
  Using cached python-docx-0.8.11.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentenc

  Using cached alembic-1.10.2-py3-none-any.whl (212 kB)
  Using cached Flask-2.2.3-py3-none-any.whl (101 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:0000:0100:01
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached databricks-cli-0.17.5.tar.gz (82 kB)
  Preparing metadata (setup.py) ... done
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.5/436.5 kB 9.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 8.4 MB/s eta 0:00:0000:0100:01m
  Using cached docker-6.0.1-py3-none-any.whl (147 kB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Using cached monotonic-1.6-py2.

# Part 1

This part is dedicated for preprocessing collections library documentation\
The original can be found here https://docs.python.org/3/library/collections.html

In [2]:
from pprint import pprint
from tqdm.auto import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

/Users/khumoyunaminaddinov/opt/anaconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-21 16:50:57.208757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import re

def extract_text(d, text_list):
    if isinstance(d, dict):
        if "document" in d and isinstance(d["document"], dict):
            extract_text(d["document"], text_list)

        if "section" in d and isinstance(d["section"], list):
            extract_text(d["section"], text_list)
        if "section" in d and isinstance(d["section"], dict):
            extract_text(d["section"], text_list)

        if "paragraph" in d and isinstance(d["paragraph"], list):
            extract_text(d["paragraph"], text_list)
        if "paragraph" in d and isinstance(d["paragraph"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["paragraph"].strip()))
        if "paragraph" in d and isinstance(d["paragraph"], dict):
            extract_text(d["paragraph"], text_list)

        if "@names" in d and isinstance(d["@names"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["@names"].strip()))

        if "#text" in d and isinstance(d["#text"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["#text"].strip()))
        if "desc_content" in d and isinstance(d["desc_content"], dict):
            extract_text(d["desc_content"], text_list)
        if "desc_name" in d and isinstance(d["desc_name"], dict):
            extract_text(d["desc_name"], text_list)
#         if "literal_block" in d and isinstance(d["literal_block"], list):
#             extract_text(d["literal_block"], text_list)
        if "desc_signature" in d and isinstance(d["desc_signature"], dict):
            extract_text(d["desc_signature"], text_list)
        if "desc" in d and isinstance(d["desc"], list):
            extract_text(d["desc"], text_list)
        if "desc_content" in d and isinstance(d["desc_content"], dict):
            extract_text(d["desc_content"], text_list)

    elif isinstance(d, list):
        for item in d:
            extract_text(item, text_list)
    elif isinstance(d, str):
        text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d.strip()))

In [4]:
import json

collections_json = json.loads(open('../data/collections.json', 'r').read())
documents = []
extract_text(collections_json, documents)
open('output.txt', 'w').write(str(documents))
len(documents)

96

In [5]:
launch_es()

In [6]:
docs = [{"content": x} for x in documents]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

/Users/khumoyunaminaddinov/opt/anaconda3/envs/chatbot/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/khumoyunaminaddinov/opt/anaconda3/envs/chatbot/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Using sep_token, but it is not set yet.


In [7]:
questions = []
answers = []

In [8]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(document_store):

    # print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = qag_pipeline.run(documents=[document])
    # print_questions(result)

    for i, ans in enumerate(result['answers']):
        if ans:
            answers.append(ans)
            questions.append(result['queries'][i])

Inferencing Samples:   0%|                                                     | 0/1 [00:00<?, ? Batches/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05 Batches/s]


Inferencing Samples: 100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.40s/ Batches]


In [9]:
# check if there is 1 answer per question
sum([len(x) for x in answers]) == len(answers)

True

In [10]:
answers = [x[0].answer for x in answers]
answers

['various approaches to working with chained maps.',
 'Example',
 'default values',
 'ChainMap',
 'the  first mapping in the chain',
 'lookups',
 'a subclass that  updates keys found deeper in the chain',
 'chainmap\\',
 'ChainMap class',
 'ChainMap class',
 'The class',
 'templating',
 'chainmap\\ objects',
 'A counter tool',
 'rich comparison operators',
 'rich comparison operators',
 'Common patterns',
 'counters that have counts greater than zero',
 'corresponding counts',
 'corresponding counts',
 'inputs with signed  counts',
 'adding an empty counter  or subtracting from an empty counter',
 'counter\\ objects',
 'This section shows various approaches to working with deques.',
 'Bounded length',
 'to maintain a sequence of recently  added elements by appending to the right and popping to the left',
 'by appending to the right and popping to the left',
 'A round-robin scheduler',
 'Values',
 'rotate()',
 'rotate() method',
 'the rotate() method',
 'reverse the rotation',
 'extend(

In [11]:
len(questions), len(answers)

(141, 141)

# Part 2

### Train DecoderGPT
This we can use for text generation with clean, consistent context

In [12]:
data = ' '.join(documents)
len(data)

13573

In [13]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(''.join(chars))
print(len(chars))

 !'(),-./012357:;<=>ABCDEFHILMNOPRSTUVW[\]_abcdefghijklmnopqrstuvwxyz’
70


#### DATA PRE-PROCESSING

In [14]:
#Converting string ids
char2ids = {ch:i for i,ch in enumerate(chars)} # This takes the characters and map characters to ids
ids2char = {i:ch for i,ch in enumerate(chars)} # This takes IDs to characters
encode = lambda s: [char2ids[c] for c in s] #Encodes a string of text
decode = lambda s: [ids2char[id] for id in s] # Decodes a string of intergers

#Checking it on some random string
''.join(decode(encode("Hi my name is Randall")))

'Hi my name is Randall'

In [15]:
''.join(decode(encode("hi there")))

'hi there'

In [16]:
import torch
data = torch.tensor(encode(data),dtype=torch.long)
print(data.shape,data.type)
print(data[:150])

torch.Size([13573]) <built-in method type of Tensor object at 0x206062570>
tensor([35, 50, 51, 61,  0, 61, 47, 45, 62, 51, 57, 56,  0, 61, 50, 57, 65, 61,
         0, 64, 43, 60, 51, 57, 63, 61,  0, 43, 58, 58, 60, 57, 43, 45, 50, 47,
        61,  0, 62, 57,  0, 65, 57, 60, 53, 51, 56, 49,  0, 65, 51, 62, 50,  0,
        45, 50, 43, 51, 56, 47, 46,  0, 55, 43, 58, 61,  7,  0, 24, 66, 43, 55,
        58, 54, 47,  0, 57, 48,  0, 61, 51, 55, 63, 54, 43, 62, 51, 56, 49,  0,
        32, 67, 62, 50, 57, 56, 69, 61,  0, 51, 56, 62, 47, 60, 56, 43, 54,  0,
        54, 57, 57, 53, 63, 58,  0, 45, 50, 43, 51, 56, 15,  0, 24, 66, 43, 55,
        58, 54, 47,  0, 57, 48,  0, 54, 47, 62, 62, 51, 56, 49,  0, 63, 61, 47,
        60,  0, 61, 58, 47, 45])


In [17]:
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [18]:
block_size= 8
train[:block_size+1]

tensor([35, 50, 51, 61,  0, 61, 47, 45, 62])

In [19]:
#How we will input our case:
x = train[:block_size]
y = train[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"when the input is {context} target is {target}")

when the input is tensor([35]) target is 50
when the input is tensor([35, 50]) target is 51
when the input is tensor([35, 50, 51]) target is 61
when the input is tensor([35, 50, 51, 61]) target is 0
when the input is tensor([35, 50, 51, 61,  0]) target is 61
when the input is tensor([35, 50, 51, 61,  0, 61]) target is 47
when the input is tensor([35, 50, 51, 61,  0, 61, 47]) target is 45
when the input is tensor([35, 50, 51, 61,  0, 61, 47, 45]) target is 62


In [20]:
#This samples batches for training. Using batch size of 8
batch_size = 4

def get_batch(split):
  data = train if split == 'train' else val
  idx = torch.randint(len(data)-block_size,(batch_size,))
  print(idx)
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  return x,y

xb,yb = get_batch('train')
print("xb shape is", xb.shape)
print("yb shape is", yb.shape)


tensor([ 9051,    97,  6510, 12144])
xb shape is torch.Size([4, 8])
yb shape is torch.Size([4, 8])


#### MODEL

In [21]:
import torch.nn as nn
from torch.nn import functional as F

In [22]:
n_embed = 32
#head_size = 8
n_heads = 4

#### SELF ATTENTION PART

In [23]:
#IN OUR IMPLEMENTATION , WE JUST USED DECODER BLOCK INSTEAD OF ENCODER

#THIS IS ESSENTIALLY QUERIES @ KEYS @ VALUES
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    #self.head_size = head_size
    self.key = nn.Linear(n_embed,head_size,bias=False)
    self.query = nn.Linear(n_embed,head_size,bias=False)
    self.value = nn.Linear(n_embed,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones((block_size,block_size))))

  def forward(self,x):
    B,T,C = x.shape
    #KEYS AND QUERIES ACT AS LINEAR TRANFORMATIONS BEFORE COMPARING OUR TOKENS TO EACH OTHER
    k = self.key(x) #dim=(B,T,16)
    q = self.query(x) #dim=(B,T,16)
    #print("key", k)
    #print("query", q)
    weights =  q @ k.transpose(-2,-1) * C**-0.5# (B,T,16) @ (B,16,T){after transposing} = (B,T,T) #THIS MEASURES THE 'ATTENTION FOR EACH TOKEN TO ANOTHER'

    weights = weights.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    #print("weights tensor", weights)
    #weights = weights.masked_fill(tril==0,float('inf'))
    weights = F.softmax(weights,dim=-1)
    #print("weights tensor", weights)
    v = self.value(x)
    output = weights @ v #(B,T,T) @ (B,T,32) =
    return output

In [24]:
#LETS DO MULTI HEAD ATTENTION
class MultiHead(nn.Module):

  def __init__(self,num_heads,head_size):
    super().__init__()
    self.multihead = nn.ModuleList([Head(head_size) for i in range(num_heads)])
    self.proj = nn.Linear(n_embed,n_embed)

  def forward(self,x):
    out =  torch.cat([h(x) for h in self.multihead],dim=-1)
    return self.proj(out)

In [25]:
class Feedforward(nn.Module):

  def __init__(self,n_embed):
    super().__init__()
    self.ffn = nn.Sequential(nn.Linear(n_embed,4*n_embed),nn.ReLU(),nn.Linear(4*n_embed,n_embed))

  def forward(self,x):
    return self.ffn(x)


In [26]:
class Block(nn.Module):

  def __init__(self,n_embed,n_heads):
    super().__init__()
    head_size = n_embed // n_heads
    self.multihead = MultiHead(n_heads,head_size)
    self.ffn = Feedforward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self,x):
    x = x + self.multihead(self.ln1(x))
    return x + self.ffn(self.ln2(x))

In [27]:
## Using the Attention Model For taking a word at time step t, and it should take the elements from 1 all the way to time step t
class Model(nn.Module):

  def __init__(self):
    super().__init__()
    self.embed = nn.Embedding(vocab_size,n_embed) #dim=(B,T,n_embed)
    self.lin = nn.Linear(n_embed,vocab_size) #dim=(B,T,Vocab_size)
    self.pos_embed = nn.Embedding(block_size,n_embed)
    self.blocks = nn.Sequential(
        Block(n_embed,n_heads=4),
        Block(n_embed,n_heads=4),
        Block(n_embed,n_heads=4)
    )
  def forward(self,x,target=None):
    B , T = x.shape
    #print(T)
    x = self.embed(x)
    #print("Dim of tokens embedding is ",x.shape)
    pos_embed = self.pos_embed(torch.arange(T))
    #print("dim of position embedding is ", pos_embed.shape)
    x = x + pos_embed
    #print("after embedding", x)
    x = self.blocks(x)
    #print("After head", x)
    #print("final dim of x is ",x.shape)
    x = self.lin(x)

    if target == None:
      loss = None
    else:
      B,T,C = x.shape
      x = x.view(B*T,C)
      target = target.view(B*T)
      loss = F.cross_entropy(x,target)

    return x, loss

  @torch.no_grad()
  def predict(self,idx,max_new_tokens):
      #idx is (batch,context/T)
    for _ in range(max_new_tokens):
      idxc = idx.clone()
      idxc = idx[:,-block_size:]
      logits , loss = self(idxc)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)

      #draw samples using multinomial
      idx_next = torch.multinomial(probs,num_samples=1)
      #print(idx)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

In [28]:
m = Model()
x , loss = m(xb,yb)
print(loss)

tensor(4.9001, grad_fn=<NllLossBackward0>)


In [29]:
## TRYING OUT
''.join(decode(m.predict(torch.zeros((1,1),dtype=torch.long),500)[0].tolist()))

" ,,klB![R,d’S’yVgEu!RgAp[(IxR:eAnm(7e=IFV0hgkedzUk<jOHy, V ey[Fqh[FgAN; WaB;OxNg.gVe_OL0wD’wewsmyh(VpUIju’T[bOv0( 1r(Vh\\mEtxF7F_>=kOLgSTLIguEx’/gwTpOHgcVcj)rys\\rCRdSA/eCTnPO[mggwxSegOyT>eWwL7sxg,HdOgyVx;seeoxjdVL:p[\\ ,LTO:ewd'll2=[7rDRS’hu(rgPm1RS,(oe=V gwWOee(:eOg’Ex=i,kFC/a3,kLxg:eA[\\A(kpHEIxlhdg rIg(qkS mHnCeFLVLeA[o\\R(wW=wtW_’=n-eg_,djOOH g,WbiA’]e!j/kvRIe\\V_PmeknHRyeo!cfLd :e=OO(WkT(_]yH;HhVxyw2hH RSoRRnVC2:ogeegq[mIRk] o7v[ev/k(lOul/jrVgkRhku(7LIW(Myg(<uumskl1b7(\\(7qd(Ie’!,BeNwle0Iky7kfy/A."

In [30]:
#create optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [31]:
batch_size = 32

for steps in range(10000):

  xb,yb = get_batch('train')
  logits, loss = m(xb,yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print(loss.item())

tensor([ 5657,   848, 11286,  6360,  8468, 10662, 10544,  9986,  9587,  8599,
         4773,  8984,  4926, 12112,  6851, 11388,  3304,  3579,  5083, 11370,
         7530,  7892,  7466,  6043,  9660,  4206, 10459,  1405,  3889,  8243,
         4952,  5910])
tensor([ 4435,  9410,  1473,  4726,  2687,  9584,   764, 10567,  5149,   218,
         7305,  4251,  5994,   173,   919,  6806,  8195,  9282,  2562, 10640,
         9239,   500,  3065,  1994,  7707,  9513, 10535,  9475,  1525,  2781,
        12005,  1882])
tensor([  665,  2784,  8580,  4214,  7537,  3623,  5189,  2611,  6872,   183,
        10122, 10628,  8772,  1765,  2486,  8315,  9711,  6557, 10307,  1280,
         1414,  5971,  7136,  7839, 10100,  3848,   143,  8499,  4569, 10345,
         6145,  1228])
tensor([ 7048,  8978,  6708,  9413,   867,  9506,  2894,  4522, 11087,  1592,
        10834,  5188,  3373,  5849, 11288,  3205, 11036, 11544,  1909, 11053,
         6459,  1930,  1200, 11286,  2070, 10446, 11857,  7100,  6659,   

tensor([ 7421,  3844,  3727, 11336,  1960,  2379,  5332,  4861,  3516,   845,
        10291,  1469, 11482,  8905,  4693,  7691,  6518,  2437, 11314,  9484,
         9748,  8549,  8415, 10178,  6190,  4696, 10979,   119,  3530, 11603,
        11718,   678])
tensor([10318,  4811,  6708, 10160,  4798,  8514,  2392,  1450,   554,  6459,
         4665,  1195,   913,   990,  3770,  7988, 10105,  9889,  2194,  9025,
         9278,  9275,  2843,  8764,  7793, 11864, 11761, 11959,  4129,  3830,
         9413,  1502])
tensor([ 4390, 10028,  1119,  7042, 10929,  2719,  8844,  3721,  6503,  5493,
         9911,  5607, 10506,  6622,  2837,  9686,   486,  2831, 12086,  2687,
         7225,  9637,  6531,  5368,  2503,  7585,  8062,  1481,  3103,  6633,
         9842,   538])
tensor([ 3630,  5454,  4007,  2003, 10498,  4018,  9066,  3255,  9006,   470,
         5489,  3491,  1595,  1036,  8702,  4472,  9092, 11473,  8064,  9650,
         5835,  4347,  3112, 11826,  5478,  6588,  4201,  3465,   725,  9

tensor([ 2530,  3658,  2598,  7265,  8815,  1863,  9104,  5425, 11672, 11713,
         4138,  6518,  2142,  1496, 10017,  8755,  3640,  2582,  3007,  4223,
         9001,  1354,   218,  1769,  3172,  8636,  9893,  2811, 10947, 11036,
         7739,  4032])
tensor([11334,  7034,  9007,   615,  1888,  3635,  2514, 10012,  5163, 10335,
         7975, 11465, 11526,  4882,  5355, 11696,  8308,  7578,  6906,  7394,
         9010,  3824,  6156,  8723,  6568,  9369, 11976,  7628,  4568,  4242,
         5357,  6431])
tensor([ 6092,  7254,  3539, 11636, 10063,  4478,    74,  5738,  8621,  1186,
         8850, 10512,   150,  3689,  3066,  6898,  5953,  6528, 11376,  4663,
         4772,  1922,  1443,  9524, 10249, 10710,  8987,  6255,  6852,  6014,
         2885,  5868])
tensor([ 9589, 10784,  8198,  1263,  6253, 12144,  1961,  7964,  6255,  7734,
        10560,  8383,  4118,  5558,  6972,  5552, 11934,  5666,  9740,  6208,
         5280,  6025,  3917,  1946,  2384,  2060,  1763,  6851,  3111,  6

tensor([ 9769,  8603,  2704,  6070,  6264,  7115,  3725,  7405,  1764, 10865,
         2260,  4212,  6881,  5627,  8040, 10547,  5112,   529,  4125, 10507,
         6922,  1134,  2683,  3923,   200,  5642,  8991,  6606,  7205,  1499,
          682, 11434])
tensor([ 5831,  2445,  2042,  2460,   978,  2939, 12165,  3478,  2159, 11857,
         5206, 10702,  5139,  6977,  7499,  9101,  6889,  3835, 11324,  5566,
        11764, 11619,  7898,  5022, 10580,  4545,  4178,  2185,  3511, 10597,
         5949,  5704])
tensor([ 6799, 10827,  3628,   774,    54, 12189,  8728,  1076,  3697,   928,
         4094,  2678, 11931,  3136,  8870, 10470,  4047,  6100,  3780, 10445,
         4050,  9725, 11329,  3657,  6059,  4847, 10379,  5967,  3043, 10047,
         4075,   460])
tensor([ 1738,   723,  8960, 11198,  2880,  7277,  1872,  9302,  3025, 10647,
        11599,  2976,  6793,  9767,  4418,  7706,  1006,  4683,  3425,  8419,
         8434, 11788,  2475, 11127,  5137,  5431,  4172,  9891, 11344,  6

tensor([  897,  9326, 10115,   805, 12117,  5168,  6525,  8211, 11391,  3890,
         7722,  8806,  4271,  7529, 11652, 11478,  1406,  2471,  7666,  8080,
         5448,  6619,  4192, 11569,  4009,  1566,  2065,   242, 10448,  3360,
         6482,  1697])
tensor([ 3746,  3633,  3099,  4910,  1818,  6593,  7603,  7532,  2436,  1576,
        11930,  5074,  2503,  6063,  5227, 11815,  5585,  4395, 11362,  4375,
         7306,  4318,  7205,  5673,  4612,    55,  9622,  5344,  7711, 10175,
         5910,  3969])
tensor([ 8832,  5643,  9244, 11011,   112,  6474,  4346,  2229,  6496,  4100,
         6567,  4618, 11901,  7956,  7182, 10251,  6700,  1813,  7329,  8299,
         8542,  9756,  9235,  3570,  6591,  6886,  1406,  7791,  3482, 10764,
         6811,  7051])
tensor([ 5040,  6567,  5049,  8098,   936, 10890, 10406,  5270, 11954,  2918,
         8570, 11179,  5092, 10191,  8212,   141,   611,  2328,  5489,  6557,
        11303,   617,  9525,  6770, 10601, 10114,  2016,  1219,  8791, 11

...

tensor([ 4954,  1732,  8967, 10002,   891, 11783,   618,  7145,  9815, 10233,
         5472,  4933,  7786,  1117,  2452,  6779,    28,  6198, 10910,  1140,
         4461,  6949, 10442,  4250,  6578, 11392,  2977, 11713,  1327,  3057,
        10020,    57])
tensor([ 6985,  8345, 11234,  6422,  8278,  2916,  5003,  1439,  9719, 10555,
          683,  2409,  4719, 11108,  7789,  3016,  2374,  7551,  1628,  1951,
         1378,   178, 10980,  4725,  1786, 10248, 11921,  4863,  7040,  1952,
         9823,  8638])
tensor([11156,  5692,  8808,  5909,  1773,  5327,   809,    10,  6107,  3790,
         7059,   967, 11952, 11288,  4998,  3975,  2871,  8041, 10089, 10934,
         2446, 12106, 11764, 11158,  8132,  3099,  5429,  3367,  6196,  7615,
         5951, 10815])
tensor([ 6346,  3147,  3762,  2065,  4206,  1898,  2072,  7384,  6050,  7227,
          801,  5326,  7440,  6281, 10179,  9725,  9323,  1341,    95,  7443,
         6475,  7210,  7726,    52,  1264,  1891,  2218,  7914,  4992,  6

tensor([ 5592, 10083,  7060,  3119,  4765,  8072,  1808, 10927,  4343,   761,
         9560,  2253,  7292,  5726,  7089,   670,  9872,  1394,  3881,  1649,
        10342,  1266,  4976, 11408,   886,  9796,   158,  1108,  9923,  6291,
         7826,  4528])
tensor([ 1792, 10089,  8891,  2501,  8508,  5683,   658,  4614,  2369,  9855,
         2152,  9994,  9761,  7382,  9564, 12162,  9303,  1625,  7353,  7869,
         3462,  3384,  1878,  7757, 11043,  7524, 10516, 10368,  9078,  3292,
         2106,  8620])
tensor([ 6555, 11245,  9046,  4717,  3698,  7012,  1187,  1263,  1842,  9685,
         4727,  1640,  9466,  3652,  3108, 10539, 11446,  5365,  7663,   989,
         9099,  2578, 11495,  5272,  4183,  5036,  4667,   695,  7468, 11492,
         8182,  4251])
tensor([ 2344,   976,  6762,  8059,  1939,  5800,  2099,  5382, 10412,  1652,
         7155, 11503,  4856,  9799, 10425,  2919,  5936,  8319,  6789,  5953,
         8704, 11877,   476,  3080, 11245,  5096,  6376,   617,  3074,  3

tensor([11352,  5611,  8237,  4058,  3199,  3918, 11856,   333,  3041,  4574,
          595,  3878, 11091,  2820, 10424, 10941,  8658,  3741,  5787,  9547,
         6352,  6243,  3137,  5077,  7951, 11898,  1002, 11432,  4677,  3732,
         8902, 11402])
tensor([  889,   137,  8883,  5283,  2699, 12163,  4369, 10436,  7760,  2213,
         6349,  5297,  2995,  6748,   215,  7143,   584,  6218, 10919,  4762,
         6594, 12159,  7061, 11869,  2025,  1964,  9461,  2093,    18,  5653,
         2343,  4017])
tensor([11599, 10630,  9719,  8131, 12140,  8690,  6633,  4963,  3689,  7667,
          608,   604,  3932, 12142, 11533, 11332, 11516,   993,  4740,  9894,
         2577,  1549,  9148, 11038,  7213,  3091, 10981,  9648,  9035,  8906,
         2953,  7812])
tensor([11194,  7021,  2948,  2803,  6182,  3752,  2940,  7349,  3135,  8156,
        10737,  8621,  7455,  3813,  7741,  7093,  5409,  1265,  1881, 11774,
        10810,  1095,  1737, 11133,  6127,   806,  8232,   924,  8769,  4

tensor([ 6671,  2037, 10807,  7179,  8820,  8671,  3514,  7841,  1876,  9768,
         3359,  4695,  7084,  3404,  9515,  6724,  3179,  3659,  8844,  6295,
         4860,  1178,  6461, 11025,   969, 10755,  6771,  3332, 11390, 10151,
         3283,  8010])
tensor([ 6328, 10404, 11587,  9999,  2015,  9368,  9247,  1730,  1057,   606,
         8779, 11218,  5462,  9865,  2991,  7668,  9322,  7931,  7166,   894,
         9955,  5252,  6429,  9575,  1292,  6239, 11064,  9900,  9370,  7134,
         6054,  8711])
tensor([11640,  3521,  8627,  5647,  2482,  3141,  1546, 11990, 11255,  6717,
         8194,  6598,  9755,   515,  9971,  2928,  4242,  1669,   392,  8290,
         6047, 10072,  6313,  7121,  5784,  1510,  1150,  6228,  2841,   204,
         3298,  6443])
tensor([ 9755,  9490,  9823, 10497, 12109, 12118,  8545, 11796,  6324,  2150,
         6091,  4462,  5444,  1368,  4238,  9209,  3829,  4098,  1558, 11306,
         5137, 10562,  6329, 10289,   178,  6968, 10085,  3631,  3073, 11

tensor([ 8165, 11256,  2997,  5854,   498,  5308,  6391,  2799,   175,  7325,
         5522,   994, 11155,  1001, 11863,  8078,  7953,  6662,  6741,  6873,
        10690,  6004,  9375,  7972,  5678,  5503, 11622,   137,  7099,  7404,
         4502, 11726])
tensor([ 8081,  3987,  5596,  7372,  3445, 11948,  3722,  6515, 10699,  2603,
         1329,   748, 11186, 11680, 11175,  4875,  2171,   442,  7284, 10406,
         4306, 10593,  7882, 10616,  6220,  1605,  5620, 10443,  7976,  8635,
         3656,  8738])
tensor([ 1336,  1656,  8843,  7284, 12082,  9477,   709,  4968,  5029,  9329,
         2192,  2791,  1696,   249,  7296,  9914,  8224,  7324,   547,  4413,
         4761,  6276,   333, 11711,  9666,  1757,  2826,   764,  4387, 10621,
         7482, 11327])
tensor([  620,  5980,  1556,  9724,  1567,   572,  8749,  6234, 11449,  9993,
         3822,  6353, 10984,  9508, 10260, 10884,  2924,   151,  8694,  5713,
        11366,   279, 11458,  9955, 11199, 11647,  2120, 11806,   181,  9

In [32]:
''.join(decode(m.predict(torch.zeros((1,1),dtype=torch.long),50)[0].tolist()))

' but slow bxine pexatting method which commass for '

# Part 3
### seq2seq word based (RNN)
Yes we know that RNNs have problem with long-term sequences, nevertheless we might find\
application for some parts like text summarization

In [33]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [34]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [35]:
import numpy as np

r = np.random.randint(1,len(questions))
print ('original text......')
for i in range(r, r+3):
    print(questions[i])
    print(answers[i])
    print()
print ('cleaned text......')
for i in range(r, r+3):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

original text......
What does _field_defaults do?
_field_defaults

Dictionary mapping field names to what?
default values

What is a straightforward way to create an ordered dictionary variant that remembers the order the keys were inserted?
If a new entry overwrites an existing entry, the  original insertion position is changed and moved to the end

cleaned text......
what does _field_defaults do
_field_defaults

dictionary mapping field names to what
default values

what is a straightforward way to create an ordered dictionary variant that remembers the order the keys were inserted
if a new entry overwrites an existing entry the original insertion position is changed and moved to the end



#### Selecting qns and answers with appropriate length (<20 words)

In [36]:
import pandas as pd

# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [37]:
lengths['counts'].describe()

count    282.000000
mean       6.234043
std        4.913687
min        0.000000
25%        2.000000
50%        6.000000
75%       10.000000
max       40.000000
Name: counts, dtype: float64

In [100]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

10.0
11.0
12.0
14.0
19.0


In [39]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])

print(len(short_questions))
print(len(short_answers))

76
76


In [40]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

the keyword def and the duplicate fieldname abc are eliminated by which conversion
['abc' wouldef' 'ghi' 'abc'] is converted to ['abc' '_1' 'ghi' '_3']

what must come after any fields without a default
fields with a default value

what class method makes a new instance from an existing sequence or iterable
class method



#### Preprocess for word based model

In [41]:
#choosing number of samples
num_samples = 76  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

In [42]:
#append start and end tokens for the answers
short_answers2 = []
for ans in short_answers:
    ans = '<SOS> ' + ans + ' <EOS>'
    short_answers2.append(ans)

In [43]:
# Create a dictionary for the frequency of the vocabulary
# Create
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers2:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [44]:
# Remove rare words from the vocabulary.
# We aim to replace fewer than 5% of words with <UNK>

threshold = 20
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [45]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 403
Size of vocab we will use: 9


In [46]:
#we will create dictionaries to provide a unique integer for each word.
vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        vocab_to_int[word] = word_num
        word_num += 1

In [47]:
# Add the unique tokens (pad and unknown vocab) to the vocabulary dictionaries.
codes = ['<PAD>','<UNK>']
for code in codes:
    code_int = len(vocab_to_int)
    vocab_to_int[code] = code_int

In [48]:
#switch <PAD> value to well's value of 0 for padding purposes later
print (vocab_to_int['<PAD>'])
for i, v in vocab_to_int.items():
    if v == 0:
        print (i)

9
what


In [49]:
for i, v in vocab_to_int.items():
    if v == 0:
        vocab_to_int[i] = vocab_to_int['<PAD>']
vocab_to_int['<PAD>'] = 0

In [50]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

In [51]:
# Check the length of the dictionaries.
print(len(vocab_to_int))
print(len(int_to_vocab))

11
11


In [52]:
# Convert the text to integers with paddings
# Replace any words that are not in the respective vocabulary with <UNK>
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int.append(ints)

answers_int = []
for answer in short_answers2:
    ints = []
    for word in answer.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int.append(ints)
# Check the lengths
print(len(questions_int))
print(len(answers_int))

76
76


In [53]:
# Calculate what percentage of all words have been replaced with <UNK># Calcul
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

for answer in answers_int:
    for word in answer:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

unk_ratio = round(unk_count/word_count,4)*100

print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 1259
Number of times <UNK> is used: 827
Percent of words that are <UNK>: 65.69%


In [54]:
from keras_preprocessing.sequence import pad_sequences

#include padding
encoder_input_data = pad_sequences(questions_int, maxlen=max_line_length, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length
decoder_input_data = pad_sequences(answers_int, maxlen=max_line_length+2, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length + start and end tokens

In [56]:
#decoder target is 1 timestep (word) ahead of decoder input, in a 3-d array
decoder_target_data = np.zeros(
    (len(answers_int), max_line_length+2, len(vocab_to_int)), #memory error occurs after 3500
    dtype='float32')

In [57]:
for i, target_seq in enumerate(answers_int):
    for t, seq in enumerate(target_seq):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, seq] = 1.

In [58]:
print (encoder_input_data.shape)
print (decoder_input_data.shape)
print (decoder_target_data.shape)

(76, 20)
(76, 22)
(76, 22, 11)


In [124]:
#include embedding size
embedding_size = 200

In [125]:
from keras.layers import Input, Embedding, Bidirectional, LSTM, concatenate, Dense

In [126]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(len(vocab_to_int), embedding_size)(encoder_inputs)
encoder = Bidirectional(LSTM(100, return_state=True))
encoder_outputs, state_h_1, state_c_1, state_h_2, state_c_2 = encoder(en_x)

In [127]:
state_h = concatenate([state_h_1, state_h_2], axis=1)
state_c = concatenate([state_c_1, state_c_1], axis=1)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [128]:
#non-bidirectional approach
# encoder_inputs = Input(shape=(None,))
# en_x=  Embedding(len(vocab_to_int), embedding_size)(encoder_inputs)
# encoder = LSTM(50, return_state=True)
# encoder_outputs, state_h, state_c = encoder(en_x)
# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

In [129]:
from keras.models import Model

In [130]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(len(vocab_to_int), embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(vocab_to_int), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) #sparse_categorical_crossentropy as labels in a single integer array

In [131]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 200)    2200        ['input_14[0][0]']               
                                                                                                  
 input_15 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 bidirectional_2 (Bidirectional  [(None, 200),       240800      ['embedding_8[0][0]']            
 )                               (None, 100),                                               

In [132]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [133]:
#early stopping & saving
# earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('Best_weights_collections.hdf5', save_best_only=True, monitor='val_loss', mode='min')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

In [134]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=40,
          validation_split=0.05,
          callbacks= [mcp_save]
#           callbacks=[earlyStopping, mcp_save, reduce_lr_loss] #change from 0.1)))
         )

Epoch 1/40
1/1 [==============================] - 5s 5s/step - loss: 0.6301 - acc: 0.0183 - val_loss: 0.9244 - val_acc: 0.2614
Epoch 2/40
1/1 [==============================] - 0s 116ms/step - loss: 0.6116 - acc: 0.1711 - val_loss: 0.8990 - val_acc: 0.2614
Epoch 3/40
1/1 [==============================] - 0s 112ms/step - loss: 0.5941 - acc: 0.1711 - val_loss: 0.8671 - val_acc: 0.2614
Epoch 4/40
1/1 [==============================] - 0s 110ms/step - loss: 0.5725 - acc: 0.1711 - val_loss: 0.8192 - val_acc: 0.2614
Epoch 5/40
1/1 [==============================] - 0s 125ms/step - loss: 0.5413 - acc: 0.1711 - val_loss: 0.7348 - val_acc: 0.2614
Epoch 6/40
1/1 [==============================] - 0s 110ms/step - loss: 0.4900 - acc: 0.1711 - val_loss: 0.6366 - val_acc: 0.2614
Epoch 7/40
1/1 [==============================] - 0s 88ms/step - loss: 0.4264 - acc: 0.1711 - val_loss: 0.6582 - val_acc: 0.2614
Epoch 8/40
1/1 [==============================] - 0s 119ms/step - loss: 0.4081 - acc: 0.1692 -

In [135]:
model.save('s2s_collections_40.h5')

In [136]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 200)    2200        ['input_14[0][0]']               
                                                                                                  
 bidirectional_2 (Bidirectional  [(None, 200),       240800      ['embedding_8[0][0]']            
 )                               (None, 100),                                                     
                                 (None, 100),                                                     
                                 (None, 100),                                               

In [137]:
#Create sampling model
decoder_state_input_h  = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [138]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['<SOS>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<EOS>' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [139]:
for i in range(50):
    seq_index = np.random.randint(1, len(encoder_input_data))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', short_answers[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['rotate method']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['reverse iteration']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['named tuples']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['subclass shown above']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['makes the defaultdict useful for building a dictionary of sets']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['base class']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <U

1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['rich comparison operators']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['inputs with signed counts']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['a named tuple']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['positional names']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['anywhere a regular dictionary is used']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ["['abc' wouldef' 'ghi' 'abc'] is converted to ['abc' '_1' 'ghi' '_3']"]
D

1/1 [==============================] - 0s 15ms/step
-
Input sentence: ['to use a lambda function']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['a single parameter']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['python 37']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['version 35']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['your own listlike classes']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 15ms/step
-
Input sentence: ['a named tuple']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [========

1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['the mapping']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 15ms/step
-
Input sentence: ['reverse iteration']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['dict list set and tuple']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['reverse iteration']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['default values']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: ['access the first element']
Decoded sentence:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
1/1 [=

### TODO:
- implement word based GPTDecoder
- clean data and make more sensible documents
- create test pipeline